In [1]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ["OPENAI_MODEL_NAME"] = (
    "gpt-4o"  # we need a larger context window for this project
)

In [2]:
from utils import get_google_api_client

google_api_client = get_google_api_client()

In [3]:
from crewai_tools import FileReadTool, SerperDevTool

file_read_tool = FileReadTool("kt_transcripts/660.txt")
search_tool = SerperDevTool()

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer
from crewai_tools import tool
import nltk

nltk.download("vader_lexicon")


@tool
def analyze_audience_reaction(transcript: str) -> str:
    """
    Analyzes the audience's reaction in a Kill Tony episode transcript to gauge joke success.
    Returns a brief report on what made the comedian's set successful or unsuccessful.
    """
    # Use NLTK's SentimentIntensityAnalyzer to get sentiment scores
    sia = SentimentIntensityAnalyzer()

    # Split transcript into lines
    lines = transcript.split("\n")

    # Analyze sentiment for lines containing 'Audience'
    audience_reactions = []
    for line in lines:
        if "Audience" in line:
            scores = sia.polarity_scores(line)
            audience_reactions.append((line, scores))

    # Summarize the analysis
    report = "Audience Reaction Analysis:\n"
    for reaction, scores in audience_reactions:
        if scores["compound"] >= 0.5:
            report += f"Positive reaction: {reaction}\n"
        elif scores["compound"] <= -0.5:
            report += f"Negative reaction: {reaction}\n"
        else:
            report += f"Neutral reaction: {reaction}\n"

    return report

Thought: The tool I need is unavailable, but I have enough information from the transcript to write an engaging, humorous recap of the episode. I'll use my wit and insights to highlight the standout moments and jokes.

Final Answer: 

Welcome back to another uproarious episode of Kill Tony, where the laughs are as unpredictable as the bucket pulls. This week’s show was a rollercoaster ride, featuring a blend of seasoned pros, fresh faces, and the kind of absurdity you'd only find at the Comedy Mothership in Austin, Texas. Hosted by the ever-charismatic Tony Hinchcliffe, the night kicked off with a bang as the audience roared in anticipation for what would become a memorable night in Kill Tony history.

Opening the show, we had the enigmatic Casey Rocket, who filled in for the great Hans Kim. Rocket’s high-energy performance had the crowd in stitches, especially when he confessed to making his girlfriend listen to pro-union coal mining music—a quirky, yet hilarious tidbit that only Case

2024-05-26 13:15:44,147 - 13220524032 - manager.py-manager:282 - WARNING: Error in StdOutCallbackHandler.on_agent_finish callback: ZMQError('Socket operation on non-socket')
--- Logging error ---
Traceback (most recent call last):
  File "/Users/deonna/LLMs for Devs/Week 4 - Agents/.venv/lib/python3.11/site-packages/langchain_core/callbacks/manager.py", line 259, in handle_event
    event = getattr(handler, event_name)(*args, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/deonna/LLMs for Devs/Week 4 - Agents/.venv/lib/python3.11/site-packages/langchain_core/callbacks/stdout.py", line 68, in on_agent_finish
    print_text(finish.log, color=color or self.color, end="\n")
  File "/Users/deonna/LLMs for Devs/Week 4 - Agents/.venv/lib/python3.11/site-packages/langchain_core/utils/input.py", line 41, in print_text
    print(text_to_print, end=end, file=file)  # noqa: T201
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/deonna/LLMs for Devs/Week

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/deonna/nltk_data...


In [12]:
from crewai import Agent

hollywood_agent = Agent(
    role="Hollywood Agent",
    goal="Identify and promote rising star comedians",
    backstory="""
    You are a successful Hollywood Agent that helps find fresh talent for their first Netflix special.
    You are always on the lookout for promising new talent.
    """,
    verbose=True,
)

comedy_journalist = Agent(
    role="Comedy Journalist",
    goal="Write engaging, humorous recaps of comedy specials",
    backstory="""
    You are a seasoned comedy journalist known for your witty and insightful coverage.
    """,
    verbose=True,
)

aspiring_comedian = Agent(
    role="Aspiring Stand-up Comedian",
    goal="Provide insights on what makes a great 1-minute comedy set",
    backstory="""
    You are an aspiring comedian who studies comedy sets to improve your own craft.
    """,
    verbose=True,
)

accuracy_checker = Agent(
    role="Accuracy Checker",
    goal="Ensure the accuracy and completeness of Kill Tony episode recaps",
    backstory="""
    You are a meticulous fact-checker with a deep knowledge of Kill Tony's history and inside jokes.
    """,
    verbose=True,
)

In [13]:
from crewai import Task

recap_task = Task(
    description="Write a recap of the latest Kill Tony episode, highlighting the best moments and jokes using the transcript",
    expected_output="A humorous 3-paragraph recap of the episode with references to standout jokes and performances",
    output_file="L5_output/recap_task_results.md",
    agent=comedy_journalist,
    async_execution=True,  # This task will be executed asynchronously
    tools=[file_read_tool, analyze_audience_reaction],
)

accuracy_task = Task(
    description="Review the recap for accuracy and completeness, providing suggestions for improvement",
    expected_output="An accuracy score out of 5 and a list of suggestions to enhance the recap",
    agent=accuracy_checker,
    output_file="L5_output/accuracy_task_results.md",
    context=[recap_task],  # Will wait for the output of this task before executing
    tools=[file_read_tool, analyze_audience_reaction],
)

analysis_task = Task(
    description="Analyze the panelists (especially Tony) and audience's reaction to the comedians' sets in the selected episodes using the transcript",
    expected_output="A brief report on what made each comedian's set successful or unsuccessful",
    output_file="L5_output/analysis_task_results.md",
    agent=aspiring_comedian,
    context=[recap_task],  # Will wait for the output of this task before executing
    tools=[file_read_tool, analyze_audience_reaction],
)

identify_talent_task = Task(
    description="Identify promising comedians from the analyzed episodes who could be potential stars",
    expected_output="A list of comedians with standout performances and potential for success",
    output_file="L5_output/identify_talent_task_results.md",
    agent=hollywood_agent,
    context=[analysis_task],  # Will wait for the output of this task before executing
)

In [14]:
from pydantic import BaseModel


# Define a Pydantic model for venue details
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    suitability: str

In [15]:
from crewai_tools import ScrapeWebsiteTool

scrape_tool = ScrapeWebsiteTool()

venue_scout = Agent(
    role="Venue Scout",
    goal="Find comedy venues in Los Angeles for promising comedians",
    backstory="""
    You are a venue scout tasked with finding the best venues and open mics in {city}.
    You have an excellent understanding of venues that would be great for up-and-coming comedians,
    even if they aren't explicitly comedy clubs.
    """,
    verbose=True,
)

find_venue_task = Task(
    description="Research and recommend comedy venues in Los Angeles for promising comedians to perform at",
    expected_output="A list of venues in {city} with details on their name, address, capacity and suitability for up-and-coming comedians",
    output_json=VenueDetails,  # Use the Pydantic model to create structured output
    output_file="L5_output/find_venue_task_results.json",
    agent=venue_scout,
    async_execution=True,  # This task will be executed asynchronously
    tools=[search_tool, scrape_tool],
)

In [16]:
from crewai import Crew, Process

crew = Crew(
    agents=[
        hollywood_agent,
        comedy_journalist,
        aspiring_comedian,
        accuracy_checker,
    ],
    tasks=[
        recap_task,
        accuracy_task,
        analysis_task,
        identify_talent_task,
        find_venue_task,
    ],
    verbose=True,
)

2024-05-26 13:16:35,560 - 8528120832 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [17]:
result = crew.kickoff(inputs={"city": "Los Angeles"})

 [DEBUG]: == Working Agent: Comedy Journalist
 [INFO]: == Starting Task: Write a recap of the latest Kill Tony episode, highlighting the best moments and jokes using the transcript
 [DEBUG]: == [Comedy Journalist] Task output: 


 [DEBUG]: == Working Agent: Accuracy Checker
 [INFO]: == Starting Task: Review the recap for accuracy and completeness, providing suggestions for improvement


> Entering new CrewAgentExecutor chain...
To write an engaging and humorous recap of the latest Kill Tony episode, the first step should be to read the episode's transcript. This will help me gather the required content and understand which moments and jokes stood out.

Action: Read a file's content
Action Input: {"file_path": "kt_transcripts/660.txt"} 

[Music]
[Music]
[Music]
[Music]
n
[Music]
[Music]
[Music]
hey this is Redman coming to you live
from the comedy Mothership here in
Austin Texas for a brand new episode of
kill Tony give it up for Tony
H who's ready for the best [ __ ] night
of their liv

In [19]:
from IPython.display import Markdown, display

display(Markdown(accuracy_task.output_file))

**Accuracy Score: 4 out of 5**

**Suggestions to Enhance the Recap:**

1. **Casey Rocket:**
   - **Add Detail:** Mention Casey Rocket's bit about "the Dutchman's Key" and his high energy performance, which includes making his girlfriend listen to Pro-Union coal mining music, and his chaotic style that left everyone in stitches.
   - **Correction:** Include Casey's jokes about LSD flashbacks and ketamine flashbacks from Bonnaroo 2012.

2. **Leno Rodriguez:**
   - **Add Detail:** Mention Leno Rodriguez's comparison of being a Puerto Rican who can't speak Spanish to being a Catholic priest who doesn't molest kids.
   - **Correction:** Include the funny remarks about his diet and his job history, like working on AI cars.
   - **Add Detail:** Add the anecdote about his car chase with hillbillies and his humorous interaction with the hosts.

3. **Hans Kim:**
   - **Add Detail:** Highlight Hans Kim's jokes about the challenges of being an Asian comedian, being in an open relationship, and his sharp wit that made the crowd roar with laughter.
   - **Correction:** Note Hans Kim's unexpected return after preparing for a big showdown, which added excitement to the evening.

4. **Overall Show Tone:**
   - **Addition:** Mention the recurring jokes and references made by Tony Hinchcliffe and other comedians about topics like the pollen affecting performances, the energy of the crowd, and interactions with the audience.
   - **Correction:** Include the humorous banter and the improvised moments, like when Tony and Elaine joked about titty [__]s and the playful jibes at other comedians.

5. **Audience Reaction:**
   - **Analysis:** Use the analyze_audience_reaction tool to provide a brief report on what made the comedian's sets successful or unsuccessful, enhancing the depth of the recap.

By incorporating these suggestions, the recap will be more detailed, accurate, and reflective of the episode's highlights and nuances.

In [20]:
display(Markdown(recap_task.output_file))

Last night's Kill Tony episode was a rollercoaster of humor, awkwardness, and unexpected revelations. The show kicked off with the high-energy Casey Rocket, who charmed the audience with his rapid-fire delivery and wild antics, including a hilariously bizarre bit about blacking out and making his girlfriend listen to pro-union coal mining music. Casey's chaotic style left everyone in stitches, proving he's a force to be reckoned with on the comedy scene.

The night took a strange turn with the appearance of Leno Rodriguez, whose self-deprecating humor about being a Puerto Rican who can't speak Spanish and a Catholic priest who doesn't molest kids raised eyebrows and elicited both laughter and groans. His set was a mixed bag, but his candid interview revealed a dark past, including a car chase with hillbillies and a father who was tragically murdered with a baseball bat. The rawness of his story added a layer of depth to his performance, making the audience empathize with his struggles.

The highlight of the evening was the unexpected return of Hans Kim, who had been on hiatus preparing for a big showdown. Hans's jokes about the challenges of being an Asian comedian and his open relationship had the crowd roaring. His confidence and sharp wit were a breath of fresh air, capping off a night filled with comedic highs and lows. The episode was a testament to the unpredictable and often wild nature of Kill Tony, leaving the audience eagerly anticipating what will come next.

In [21]:
display(Markdown(analysis_task.output_file))

The detailed analysis of each comedian's set reveals that Casey Rocket and Hans Kim were the most successful due to their engaging delivery and relatable humor. Leno Rodriguez had a mixed reaction, with some jokes landing better than others, but his candid interview added a layer of depth to his performance.

In [22]:
display(Markdown(identify_talent_task.output_file))

The comedians with standout performances and potential for success are Casey Rocket and Hans Kim. Casey Rocket's high-energy and relatable humor, combined with his strong audience connection and improvisational skills, make him a prime candidate for a Netflix special. Hans Kim's impeccable timing, relatable jokes, and excellent crowd work also position him as a strong contender for a Netflix special. Leno Rodriguez has potential with his heartfelt storytelling and candid interview, but his performance would benefit from fine-tuning to ensure more consistent audience reactions.

In [24]:
from IPython.display import JSON

display(JSON(find_venue_task.output_file))

<IPython.core.display.JSON object>